In [ ]:
# Import packages from the Python standard library
import importlib.util
import os
import sys
import pprint
import time
import warnings
from pathlib import Path


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name
EXPERIMENT_NAME = "mnist"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:5000"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

#from scripts.client import DioptraClient
from scripts.utils import make_tar

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

In [ ]:
from __future__ import annotations

import os
from pathlib import Path
from posixpath import join as urljoin
from typing import Any
from urllib.parse import urlparse, urlunparse
import inspect
import requests
from typing import Any, cast

import structlog
from structlog.stdlib import BoundLogger

LOGGER: BoundLogger = structlog.stdlib.get_logger()

class APIConnectionError(Exception):
    '''Class for connection errors'''
class StatusCodeError(Exception):
    '''Class for status code errors'''
class JSONDecodeError(Exception):
    '''Class for JSON decode errors'''

def create_data_dict(**kwargs):
    return kwargs

def debug_request(url, data=None):
    LOGGER.debug("Request made.", url=url, data=data)
def debug_response(json):
    LOGGER.debug("Response received.", json=json)

def get(session, endpoint, *features):
    debug_request(urljoin(endpoint, *features))
    return make_request(session, 'get', endpoint, None, *features)
def post(session, endpoint, data, *features):
    debug_request(urljoin(endpoint, *features), data)
    return make_request(session, 'post', endpoint, data, *features)
def delete(session, endpoint, data, *features):
    debug_request(urljoin(endpoint, *features), data)
    return make_request(session, 'delete', endpoint, data, *features)
def put(session, endpoint, data, *features):
    debug_request(urljoin(endpoint, *features), data)
    return make_request(session, 'put', endpoint, data, *features)

def make_request(session, method_name, endpoint, data, *features):
    url = urljoin(endpoint, *features)
    method = getattr(session, method_name)
    try: 
        if (data):
            response = method(url, json=data)
        else:
            response = method(url)
        if response.status_code is not 200:
            raise StatusCodeError() 
        json = response.json()
    except (requests.ConnectionError, StatusCodeError, requests.JSONDecodeError) as e:
        handle_error(session, url, method_name.upper(), data, response, e) 
    debug_response(json=json)
    return json

def handle_error(session, url, method, data, response, error):
    if (type(error) is requests.ConnectionError):
        restapi = os.environ["DIOPTRA_RESTAPI_URI"]
        message = f"Could not connect to the REST API. Is the server running at {restapi}?"
        LOGGER.error(message, url=url, method=method, data=data, response=response.text);
        raise APIConnectionError(message);
    if (type(error) is StatusCodeError):
        message = f"Error code {response.status_code} returned."
        LOGGER.error(message, url=url, method=method, data=data, response=response.text);
        raise StatusCodeError(message);
    if (type(error) is requests.JSONDecodeError):
        message = f"JSON response could not be decoded."
        LOGGER.error(message, url=url, method=method, data=data, response=response.text);
        raise JSONDecodeError(message);

        

class DioptraClient(object):
    def __init__(self, session, address=None, api_version="v1") -> None:
        self._session = session
        self._users = UsersClient(session, "users", address, api_version)
        self._auth = AuthClient(session, "auth", address, api_version)
        self._queues = QueuesClient(session, "queues", address, api_version)
        self._groups = GroupsClient(session, "groups", address, api_version)
        self._tags = TagsClient(session, "tags", address, api_version)
        self._plugins = PluginsClient(session, "plugins", address, api_version)
        self._pluginParameterTypes = PluginParameterTypesClient(session, "pluginParameterTypes", address, api_version)
        # experiments
        # models
        # jobs
        # artifacts
        # entrypoints
    @property
    def users(self):
        return self.get_endpoint(self._users)
    @property
    def auth(self):
        return self.get_endpoint(self._auth)
    @property
    def queues(self):
        return self.get_endpoint(self._queues)
    @property
    def groups(self):
        return self.get_endpoint(self._groups)
    @property
    def tags(self):
        return self.get_endpoint(self._tags)
    @property
    def plugins(self):
        return self.get_endpoint(self._plugins)
    @property
    def pluginParameterTypes(self):
        return self.get_endpoint(self._pluginParameterTypes)

    def get_endpoint(self, ep):
        ep.session = self._session
        return ep

class Endpoint(object):
    def __init__(self, session, ep_name, address, api_version) -> None:
        address = (
            f"{address}/api/{api_version}"
            if address
            else f"{os.environ['DIOPTRA_RESTAPI_URI']}/api/{api_version}"
        )
        self._scheme, self._netloc, self._path, _, _, _ = urlparse(address)
        self._ep_name = ep_name
        self._session = session
    @property
    def session(self):
        return self._session
    @session.setter
    def session(self, s):
        self._session = s
    @property
    def url(self):
        return self.def_endpoint(self._ep_name)
    def def_endpoint(self, name):
        '''creates base url for an endpoint by name'''
        return urlunparse(
            (self._scheme, self._netloc, urljoin(self._path, name + "/"), "", "", "")
        )

class UsersClient(Endpoint):
    def get_all(self):
        '''gets all users'''
        return get(self.session, self.url)
    def create(self, username, email, password, confirm_password):
        '''creates a user'''
        d = {"username":username,
             "email": email,
             "password": password,
             "confirmPassword": confirm_password}
        return post(self.session, self.url, d)
    def get_by_id(self, user_id):
        '''get a user by id'''
        return get(self.session, self.url, str(user_id))
    def update_password_by_id(self, user_id, old_password, new_password, confirm_new_password):
        '''change a user's password by id'''
        d = {"oldPassword":old_password,
             "newPassword": new_password,
             "confirmNewPassword": confirm_new_password}
        return post(self.session, self.url, d, str(user_id), 'password')
    def current(self):
        '''get the current user'''
        return get(self.session, self.url, 'current')
    def delete_current(self, password):
        '''delete the current user'''
        d = {"password":password}
        return delete(self.session, self.url, d, 'current')
    def modify_current(self, username, email):
        '''modify the current user'''
        d = {"username":username,
             "email": email}
        return put(self.session, self.url, d, 'current')
    def modify_current_password(self, old_password, new_password, confirm_new_password):
        '''modify the current user's password'''
        d = {"oldPassword":old_password,
             "newPassword": new_password,
             "confirmNewPassword": confirm_new_password}
        return post(self.session, self.url, d, 'current', 'password')
    def failed_user_post(self):
        '''create a post request with an invalid schema, for testing'''
        return post(self.session, self.url, {'a':'doesnotexist'})
    def failed_user_get(self):
        '''create a get request to an invalid url, for testing'''
        return get(self.session, self.url, 'doesnotexist')

class AuthClient(Endpoint):
    def login(self, username, password):
        '''login as the given user'''
        d = {"username":username,
             "password": password}
        return post(self.session, self.url, d, 'login')
    def logout(self, everywhere):
        '''logout as the current user'''
        d = {"everywhere": everywhere}
        return post(self.session, self.url, d, 'logout')

class GroupsClient(Endpoint):
    def get_all(self):
        '''get all groups'''
        return get(self.session, self.url)
    def get_by_id(self, gid):
        '''get a group by id'''
        return get(self.session, self.url, str(gid))
        
class QueuesClient(Endpoint):
    def get_all(self):
        '''gets all queues'''
        return get(self.session, self.url)
    def create(self, group, name, description):
        '''create a queue'''
        d = {"group": group,
             "name": name,
             "description": description
            }
        return post(self.session, self.url, d)
    def get_drafts(self):
        '''gets all queue drafts'''
        return get(self.session, self.url, 'drafts')
    def create_draft(self, group, name, description):
        '''create a draft'''
        d = {"group": group,
             "name": name,
             "description": description
            }
        return post(self.session, self.url, d, 'drafts')
    def modify_draft_by_draft_id(self, draft_id, name, description):
        '''modify a draft by id'''
        d = {"name": name,
             "description": description
            }
        return put(self.session, self.url, d, 'drafts', str(draft_id))
    def delete_draft_by_draft_id(self, draft_id):
        '''delete a draft by id'''
        d = None
        return delete(self.session, self.url, d, 'drafts', str(draft_id))
    def get_draft_by_draft_id(self, draft_id):
        '''get a draft by id'''
        return get(self.session, self.url, 'drafts', str(draft_id))
    def modify_by_queue_id(self, queue_id, name, description):
        '''modify a queue by id'''
        d = {"name": name,
             "description": description
            }
        return put(self.session, self.url, d, str(queue_id))
    def delete_by_queue_id(self, queue_id):
        '''delete a queue by id'''
        d = None
        return delete(self.session, self.url, d, str(queue_id))
    def get_by_queue_id(self, queue_id):
        '''get a queue by id'''
        return get(self.session, self.url, str(queue_id))
    def modify_draft_by_queue_id(self, queue_id, name, description):
        '''modify a draft by queue id'''
        d = {"name": name,
             "description": description
            }
        return put(self.session, self.url, d, str(queue_id), 'draft')
    def create_draft_by_queue_id(self, queue_id, name, description):
        '''create a draft by queue id'''
        d = {"name": name,
             "description": description
            }
        return post(self.session, self.url, d, str(queue_id), 'draft')
    def delete_draft_by_queue_id(self, queue_id):
        '''delete a draft by queue id'''
        d = None
        return delete(self.session, self.url, d, str(queue_id), 'draft')
    def get_draft_by_queue_id(self, queue_id):
        '''get a draft by queue id'''
        return get(self.session, self.url, str(queue_id), 'draft')
    
class TagsClient(Endpoint):
    def get_all(self):
        return get(self.session, self.url)
    def create(self, name, group):
        d = {"name": name, 
             "group": group}
        return post(self.session, self.url, d)
    def delete_by_id(self, tag_id):
        d = None
        return delete(self.session, self.url, d, str(tag_id))
    def get_by_id(self, tag_id):
        return get(self.session, self.url, str(tag_id))
    def modify_by_id(self, tag_id, name):
        d = {"name": name}
        return put(self.session, self.url, d, str(tag_id))
    def get_resources_by_id(self, tag_id):
        return get(self.session, self.url, str(tag_id), 'resources')

class PluginsClient(Endpoint):
    def get_all(self):
        return get(self.session, self.url)
    def create(self, group, name, description):
        d = {"group":group, 
             "name":name, 
             "description":description}
        return post(self.session, self.url, d)
    def get_all_drafts(self):
        return get(self.session, self.url, 'drafts')
    def create_draft(self, group, name, description):
        d = {"group":group, 
             "name":name, 
             "description":description}
        return post(self.session, self.url, d, 'drafts')
    def get_draft_by_draft_id(self, plugin_id):
        return get(self.session, self.url, 'drafts', str(plugin_id))
    def modify_draft_by_draft_id(self, plugin_id, name, description):
        d = {"name": name,
             "description": description}
        return put(self.session, self.url, d, 'drafts', str(plugin_id))
    def delete_draft_by_draft_id(self, plugin_id):
        d = None
        return delete(self.session, self.url, d, 'drafts', str(plugin_id))
    def get_by_id(self, plugin_id):
        return get(self.session, self.url, str(plugin_id))
    def modify_by_id(self, plugin_id, name, description):
        d = {"name": name,
             "description": description}
        return put(self.session, self.url, d, str(plugin_id))
    def delete_by_id(self, plugin_id):
        d = None
        return delete(self.session, self.url, d, str(plugin_id))
    def get_draft_by_plugin_id(self, plugin_id):
        return get(self.session, self.url, str(plugin_id), 'draft')
    def modify_draft_by_plugin_id(self, plugin_id, name, description):
        d = {"name": name,
             "description": description}
        return put(self.session, self.url, d, str(plugin_id), 'draft')
    def create_draft_by_plugin_id(self, plugin_id, name, description):
        d = {"name": name,
             "description": description}
        return post(self.session, self.url, d, str(plugin_id), 'draft')
    def delete_draft_by_plugin_id(self, plugin_id):
        d = None
        return delete(self.session, self.url, d, str(plugin_id), 'draft')
    def get_files_by_plugin_id(self, plugin_id):
        return get(self.session, self.url, str(plugin_id), 'files')
    def create_files_by_plugin_id(self, plugin_id, filename, contents, description, *plugins):
        d = {"filename": filename,
             "contents": contents,
             "description": description,
             "tasks": [plugin.as_dict() for plugin in plugins]
            }
        return post(self.session, self.url, d, str(plugin_id), 'files')
    def delete_files_by_plugin_id(self, plugin_id):
        d = None
        return delete(self.session, self.url, d, str(plugin_id), 'files')
    def get_files_by_plugin_id_file_id(self, plugin_id, file_id):
        return get(self.session, self.url, str(plugin_id), 'files', str(file_id))
    def modify_files_by_plugin_id_file_id(self, plugin_id, file_id, filename, contents, description, *plugins):
        d = {"filename": filename,
             "contents": contents,
             "description": description,
             "tasks": [plugin.as_dict() for plugin in plugins]
            }
        return put(self.session, self.url, d, str(plugin_id), 'files', str(file_id))
    def delete_files_by_plugin_id_file_id(self, plugin_id, file_id):
        d = None
        return delete(self.session, self.url, d, str(plugin_id), 'files', str(file_id))
    def get_snapshots_by_plugin_id(self, plugin_id):
        return get(self.session, self.url, str(plugin_id), 'snapshots')
    def get_snapshot_by_plugin_id_snapshot_id(self, plugin_id, snapshot_id):
        return get(self.session, self.url, str(plugin_id), 'snapshots', str(snapshot_id))
    def get_tags_by_plugin_id(self, plugin_id):
        return get(self.session, self.url, str(plugin_id), 'tags')
    def modify_tags_by_plugin_id(self, plugin_id, ids):
        d = {"ids":ids}
        return put(self.session, self.url, d, str(plugin_id), 'tags')
    def delete_tags_by_plugin_id(self, plugin_id):
        d = None
        return delete(self.session, self.url, d, str(plugin_id), 'tags')
    def create_tags_by_plugin_id(self, plugin_id, ids):
        d = {"ids":ids}
        return post(self.session, self.url, d, str(plugin_id), 'tags')
    def delete_tags_by_plugin_id_tag_id(self, plugin_id, tag_id):
        d = None
        return delete(self.session, self.url, d, str(plugin_id), 'tags', str(tag_id))

class PluginParameterTypesClient(Endpoint):
    def get_all(self):
        return get(self.session, self.url)
    def create(self, group, name, description, structure):
        d = {"group": group,
             "name": name,
             "description": description,
             "structure": structure}
        return post(self.session, self.url, d)
    def get_by_id(self, type_id):
        return get(self.session, self.url, str(type_id))
    def modify_by_id(self, type_id, name, description, structure):
        d = {"name": name,
             "description": description,
             "structure": structure}
        return put(self.session, self.url, d, str(type_id))
    def delete_by_id(self, type_id):
        d = None
        return delete(self.session, self.url, d, str(type_id))

class PluginTask(object):
    def __init__(self, name, inputs, outputs, client):
        self.name = name
        self.inputs = inputs # expects [(name1, type1), (name2, type2) ...]
        self.outputs = outputs # expects [(name1, type1), (name2, type2) ...]
        self.client = client
    def convert_params_to_ids(self, mappings):
        '''this converts parameters to registered ids using a mapping from register_unregistered_types'''
        return [(i[0], mappings[i[1]]) for i in self.inputs], [(o[0], mappings[o[1]]) for o in self.outputs]
    def register_unregistered_types(self, group=1):
        '''checks all the types in inputs/outputs and register things that aren't registered'''
        registered_types = self.client.pluginParameterTypes.get_all() # get all registered types
        types_used_in_plugin = set([m[1] for m in self.inputs] + [m[1] for m in self.outputs]) # get all types for this plugin
        types_to_id = {}
        for registered in registered_types['data']: # add registered types to our dictionary
            types_to_id[str(registered['name'])] = str(registered['id'])
        for used in types_used_in_plugin:
            used = str(used)
            if (used not in types_to_id): # not yet registered, so register it
                response = self.client.pluginParameterTypes.create(group, used, used + " plugin parameter", structure={})
                types_to_id[used] = str(response['id'])
        return types_to_id # mapping of types to ids
    def as_dict(self, mappings=None):
        '''convert it to a dict to be sent to the RESTAPI'''
        if mappings is None:
            mappings = self.register_unregistered_types()
        ins, outs = self.convert_params_to_ids(mappings)
        return {
            "name": self.name,
            "inputParams": [{"name": param[0], "parameterType": param[1]} for param in ins],
            "outputParams": [{"name": param[0], "parameterType": param[1]} for param in outs]
        }

In [ ]:
client = DioptraClient(requests.Session())

In [ ]:
try:
    # these might fail if this has been run before
    client.users.create('testuser','testuser@gmail.com','testuserpassword','testuserpassword')
except:
    pass
try:
    client.users.create('testuser2','testuser2@gmail.com','testuserpassword','testuserpassword')
except:
    pass
client.auth.login('testuser','testuserpassword')
client.users.get_all()
client.users.current()
client.users.update_password_by_id(client.users.current()['id'],'testuserpassword','newtestuserpassword','newtestuserpassword')
try:
    client.users.modify_current('testuser','newemail@email.com') # this one should fail because we are not logged in after a password change
except:
    pass
client.auth.login('testuser','newtestuserpassword')
client.users.modify_current('testuser','newemail@email.com')
client.users.modify_current_password('newtestuserpassword','newnewtestuserpassword','newnewtestuserpassword')
try: 
    client.auth.logout(True)
except:
    pass
client.auth.login('testuser','newnewtestuserpassword')
client.users.delete_current('newnewtestuserpassword')
try: 
    client.users.failed_user_post() # if we get the schema wrong, it does return JSON, raise if not 200, but send json back to logs
except:
    pass
try:
    client.users.failed_user_get() # if we get the URL wrong, it does not return JSON
except:
    pass
client.auth.login('testuser2','testuserpassword')
client.users.get_by_id(client.users.current()['id'])


In [ ]:
try:
    client.users.create('queuemgr','queuemgr@dioptra.nccoe.nist.gov','lmnopQUEUErs','lmnopQUEUErs')
except:
    pass # ignore if user exists already
client.auth.login('queuemgr','lmnopQUEUErs')
cpu_id = client.queues.create(1, 'tensorflow_cpu2', 'for running tensorflow on a cpu')['id']
client.queues.modify_by_queue_id(cpu_id, 'tensorflow_cpu2', 'for running tensorflow on a cpu - modified')
client.queues.get_all()
gpu_draft_id = client.queues.create_draft(1, 'tensorflow_gpu', 'for running tensorflow on a gpu')['id']
client.queues.modify_draft_by_draft_id(gpu_draft_id, 'tensorflow_gpu', 'for running tensorflow on a gpu - modified')
client.queues.get_drafts()
client.queues.get_draft_by_draft_id(gpu_draft_id)
client.queues.delete_draft_by_draft_id(gpu_draft_id)
client.queues.create_draft_by_queue_id(cpu_id, 'tensorflow_cpu2', 'oops my draft deleted the description')
client.queues.get_draft_by_queue_id(cpu_id)
client.queues.modify_draft_by_queue_id(cpu_id, 'tensorflow_cpu2', 'wait go back')
client.queues.delete_draft_by_queue_id(cpu_id)
client.queues.delete_by_queue_id(cpu_id)

In [ ]:
client.tags.get_all()
client.tags.create('tag_name', 1)
client.tags.get_all()
client.tags.get_by_id(1)
client.tags.modify_by_id(1, 'new_name')
client.tags.get_resources_by_id(1)
client.tags.delete_by_id(1)
client.tags.get_all()

In [ ]:
def this_is_my_plugin_task(x, y):
    return y, x

try:
    client.users.create('pluginuser','pluginuser@dioptra.nccoe.nist.gov','pleasemakesuretoPLUGINthecomputer','pleasemakesuretoPLUGINthecomputer')
except:
    pass # ignore if user exists already
client.auth.login('pluginuser','pleasemakesuretoPLUGINthecomputer')

# first let's create a PluginTask for our plugin
#my_plugin = PluginTask(name='this_is_my_plugin_task', inputs=[('x', 'int'), ('y', 'str')], outputs=[('y', 'str'), ('x', 'int')], client=client)
my_plugin = PluginTask(name='this_is_my_plugin_task', inputs=[('x', 'int')], outputs=[('y', 'str')], client=client)
plugin_id = client.plugins.create(1,'first_plugin', 'this plugin does nothing')['id']
client.plugins.get_all()
client.plugins.get_by_id(plugin_id)
client.plugins.modify_by_id(plugin_id, 'first_plugin', 'this plugin still does nothing')
draft_id = client.plugins.create_draft(1,'draft_second_plugin', 'this plugin might do something someday')['id']
client.plugins.get_all_drafts()
client.plugins.get_draft_by_draft_id(draft_id)
client.plugins.modify_draft_by_draft_id(draft_id, 'draft_second_plugin', 'nevermind its hopeless')
client.plugins.delete_draft_by_draft_id(draft_id)
client.plugins.create_draft_by_plugin_id(plugin_id, 'first_plugin', 'this plugin could do something now!!')
client.plugins.modify_draft_by_plugin_id(plugin_id, 'first_plugin', 'this plugin could actually do something now!!')
client.plugins.get_draft_by_plugin_id(plugin_id)
client.plugins.delete_draft_by_plugin_id(plugin_id)

client.plugins.create_files_by_plugin_id(plugin_id, 'the_current_file.py', "def this_is_my_plugin_task(x, y):\n    return y x", "i hope i didn't forget a comma", my_plugin)
client.plugins.get_files_by_plugin_id(plugin_id)
client.plugins.delete_files_by_plugin_id(plugin_id)

files_id = client.plugins.create_files_by_plugin_id(plugin_id, 'the_new_current_file.py', "def this_is_my_plugin_task(x, y):\n    return y x", "i hope i didn't forget a comma", my_plugin)['id']
client.plugins.modify_files_by_plugin_id_file_id(plugin_id, files_id, 'the_new_current_file.py', "def this_is_my_plugin_task(x, y):\n    return y, x", "i forgot a comma", my_plugin)
client.plugins.get_files_by_plugin_id_file_id(plugin_id, files_id)
client.plugins.delete_files_by_plugin_id_file_id(plugin_id, files_id)
# client.plugin.get_snapshots_by_plugin_id(plugin_id)
# client.plugin.get_snapshot_by_plugin_id_snapshot_id(plugin_id, snapshot_id) # no way to create snapshots as of now?
tag_id = client.tags.create('youreit', 1)['id']
tag_id2 = client.tags.create('nuhuh', 1)['id']
client.plugins.create_tags_by_plugin_id(plugin_id, [tag_id])
client.plugins.modify_tags_by_plugin_id(plugin_id, [tag_id, tag_id2])
client.plugins.get_tags_by_plugin_id(plugin_id)
client.plugins.delete_tags_by_plugin_id_tag_id(plugin_id, tag_id)
client.plugins.delete_tags_by_plugin_id(plugin_id)
client.plugins.delete_by_id(plugin_id)

client.pluginParameterTypes.get_all()
type_id = client.pluginParameterTypes.create(1, 'bool', 'its a yes or no question', {})['id']
client.pluginParameterTypes.modify_by_id(type_id, 'bool', 'it cant be None. probably', {})
client.pluginParameterTypes.get_by_id(type_id)
client.pluginParameterTypes.delete_by_id(type_id)



In [ ]:

for i in range(0, 50):
    try:
        client.plugins.delete_by_id(i)
    except:
        pass
    try: 
        client.tags.delete_by_id(i)
    except:
        pass
    try:
        client.pluginParameterTypes.delete_by_id(i)
    except:
        pass
client.pluginParameterTypes.get_all()


In [ ]:
def filter_data_dict(data, *args):
    data.pop('self', None) # don't need self in dictionary
    for p in args:
        data.pop(p, None) # don't need any of these in dictionary
    return data
def create_user(self, myid, username, password, confirm):
    data = filter_data_dict(locals(), 'myid') # just specify the names of parameters that aren't part of data
    return data
    # requests.post( something )
print(create_user(client, 1234, 'u', 'p', 'p'))